============================================= Markov Decision Process (MDP) =============================================

#### Markov Decision Process (MDP)

This section introduces the foundation of reinforcement learning: **Markov Decision Processes (MDPs)**. MDPs are mathematical models used to describe how systems evolve over time under a sequence of decisions. They form the basis for defining RL problems and later solution methods like Value Iteration and Q-Learning. 

##### Definition of an MDP

A **Markov Decision Process (MDP)** is a tuple: $\text{MDP}: (\mathcal{S}, \mathcal{A}, T, r)$

Where:

- **$\mathcal{S}$** — set of *states* the agent can be in (e.g., positions in a gridworld).  
- **$\mathcal{A}$** — set of *actions* the agent can take at each state.  
- **$T: \mathcal{S} \times \mathcal{A} \times \mathcal{S} \to [0,1]$** — *transition function* (probability distribution): $$T(s, a, s') = P(s_{t+1}=s' \mid s_t=s, a_t=a)$$
  This encodes how likely it is to move to state $s'$ after taking action $a$ in state $s$.

  So in an MDP, even when the current state $s$ and action $a$ are fixed, the next state $s'$ is generally **not deterministic**. Instead, the environment transitions according to above probability distribution.
- **$r: \mathcal{S} \times \mathcal{A} \to \mathbb{R}$** — *reward function*: immediate reward received when taking action $a$ in state $s$.  
  Larger rewards indicate more desirable actions relative to the task.

##### Return and Discount Factor

A **trajectory** in an MDP is a sequence generated by interacting with the environment:

$$
\tau = (s_0, a_0, r_0, s_1, a_1, r_1, s_2, a_2, r_2, \dots)
$$

- **Return** of a trajectory is the sum of rewards obtained along it:
  $R(\tau) = r_0 + r_1 + r_2 + \cdots$

- To handle potentially infinite trajectories and emphasize near-term rewards, we introduce a **discount factor** $\gamma < 1$:

  $
  R(\tau) = r_0 + \gamma r_1 + \gamma^2 r_2 + \cdots = \sum_{t=0}^\infty \gamma^t r_t
  $

  - Smaller $\gamma$ biases the agent toward near-term rewards.  
  - Larger $\gamma$ (close to 1) encourages long-term planning.

The **goal in reinforcement learning** is to *find the trajectory (policy) that maximizes expected return*.

##### Discussion of the Markov Assumption

The **Markov property** means the next state depends *only* on the current state and action — not on any earlier history.

##### Why this matters

- If the system has **memory** (e.g., velocity depending on previous state), it may *not* be Markovian.
- However, we can often *augment* the state to include necessary history (e.g., current location + velocity) so that the Markov assumption holds.

This assumption simplifies modeling and algorithms, and most RL techniques rely on it. 

##### Summary

**Key takeaways:**

- An MDP is a formal way to represent **sequential decision making under uncertainty**.  
- Defined by states, actions, transition probabilities, and rewards.  
- The objective is to choose actions that maximize *expected discounted return*.  
- Markov property simplifies modeling by assuming the future depends only on the current state and action. 

##### Optional: Additional Notes

- MDPs are the backbone of RL algorithms like Value Iteration and Q-Learning.
- In full RL problems, the agent often **doesn’t know** the exact transition function; it must learn optimal actions from experience. 

==================================== Value Iteration ====================================

### Value Iteration

This section introduces **Value Iteration**, a dynamic programming method for computing the **optimal value function** in a Markov Decision Process (MDP). It is one of the foundational algorithms in reinforcement learning when the environment model (transition and reward functions) is known.

#### Stochastic Policy

In an MDP, a **stochastic policy** $\pi(a|s)$ defines a probability distribution over actions $a \in \mathcal{A}$ for each state $s \in \mathcal{S}$. The goal is to find a policy that maximizes cumulative rewards.

#### Value Function

The **value function** $V^\pi(s)$ estimates the expected *discounted return* starting from state $s$ when following policy $\pi$.

#### Action-Value Function

The **action-value function** $Q^\pi(s,a)$ gives the expected return starting from state $s$, taking action $a$, and thereafter following policy $\pi$.

#### Optimal Stochastic Policy

An **optimal policy** $\pi^*$ maximizes the expected return for all states — and corresponds to the **optimal value function** $V^*(s)$.

#### Principle of Dynamic Programming

Value iteration uses the **Bellman Optimality Equation**:

$$
V^*(s) = \max_{a\in \mathcal{A}} \left\{r(s,a) + \gamma \sum_{s'\in \mathcal{S}} P(s'|s,a)\,V^*(s') \right\}
$$

This identity expresses that the best return from $s$ equals the best immediate action value plus the discounted value of successor states. It’s derived from the **principle of dynamic programming** — optimal substructure: *the remainder of an optimal trajectory is also optimal*.

#### Value Iteration Algorithm

##### Core Idea

Value iteration treats the Bellman Optimality Equation as a set of constraints and iteratively updates the value function until convergence.
##### Update Rule

Initialize the value function $V_0(s)$ arbitrarily (e.g., zero for all $s$). Then at iteration $k$, update:

$$
V_{k+1}(s)
= \max_{a\in \mathcal{A}}\Big\{r(s,a)
+ \gamma \sum_{s'\in \mathcal{S}}P(s'\mid s,a)\,V_k(s')\Big\},\quad \forall s\in \mathcal{S}.
$$

Continue until the values converge (changes between iterations are negligible).

##### Convergence

As $k\rightarrow \infty$, the sequence $\{V_k\}$ converges to the **optimal value function** $V^*$ regardless of initialization: $V^*(s) = \lim_{k\to\infty} V_k(s)\quad \forall s.$

Once $V^*$ is found, the corresponding **optimal policy** $\pi^*$ can be extracted by choosing actions that maximize the value at each state:

$$
\pi^*(s) = \arg\max_{a\in\mathcal{A}} \left\{
r(s,a) + \gamma \sum_{s'}P(s'|s,a)\,V^*(s')
\right\}.
$$

#### Policy Evaluation & Improvement (Implicit)

Unlike policy iteration — which separates evaluation and improvement — value iteration implicitly performs both in a single update: the value function is both evaluated under the current estimate and improved by choosing the maximizing action.

#### Implementation of Value Iteration

Typical implementations loop over states and actions, computing the Bellman update (often with a convergence threshold $\theta$ to stop early). Efficient implementations store temporary values and track the largest change to detect convergence.

#### Summary

- **Value iteration** is a dynamic programming algorithm for computing the optimal value function and policy in an MDP when the model (transition and reward functions) is fully known.  
- It iteratively applies the Bellman Optimality update until values stabilize.  
- It guarantees convergence to the optimal value function as the number of iterations goes to infinity.  
- Once converged, you can derive the optimal policy by selecting actions that maximize the value function. 

In [3]:
%matplotlib inline
import random
import numpy as np
import gym
from d2l import torch as d2l

seed = 0  # Random number generator seed
gamma = 0.95  # Discount factor
num_iters = 10  # Number of iterations
random.seed(seed)  # Set the random seed to ensure results can be reproduced
np.random.seed(seed)

# Workaround for Gym 0.26+ API change (env.seed() was removed)
def make_frozen_lake_env(seed):
    """Create FrozenLake environment compatible with Gym 0.26+"""
    env = gym.make('FrozenLake-v1', is_slippery=False)
    env.reset(seed=seed)
    env.action_space.seed(seed)
    
    # Extract environment info (same structure as d2l.make_env)
    num_states = env.observation_space.n
    num_actions = env.action_space.n
    
    # Build transition and reward matrices from env.P
    P = np.zeros((num_states, num_actions, num_states))
    R = np.zeros((num_states, num_actions))
    
    for s in range(num_states):
        for a in range(num_actions):
            for prob, next_s, reward, done in env.P[s][a]:
                P[s, a, next_s] += prob
                R[s, a] += prob * reward
    
    return env, P, R, num_states, num_actions

# Now set up the environment
env, P, R, num_states, num_actions = make_frozen_lake_env(seed)
print(f"Number of states: {num_states}, Number of actions: {num_actions}")

Number of states: 16, Number of actions: 4
